In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import json
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import csv
from keras import optimizers
import keras
from functools import partial
from math import exp
from keras.utils import get_custom_objects
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report 
import joblib
import pickle
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score, roc_curve
from itertools import cycle

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Set only the first GPU as visible
        tf.config.set_visible_devices(gpus[0], 'GPU')
        # Allow memory growth to allocate memory dynamically on the GPU
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("GPU configuration successful.")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU detected.")

In [ ]:
from keras.mixed_precision import Policy
from keras.mixed_precision import set_global_policy

policy = Policy('mixed_float16')
set_global_policy(policy)

In [ ]:
def load_data(data_path):
    """Loads training dataset.

    """
    X_train = np.load(f'{data_path}/X_train.npy')
    X_test = np.load(f'{data_path}/X_test.npy')
    X_validation = np.load(f'{data_path}/X_val.npy')

    y_train = np.load(f'{data_path}/y_train.npy')
    y_test = np.load(f'{data_path}/y_test.npy')
    y_validation = np.load(f'{data_path}/y_val.npy')

    y_train = y_train[..., np.newaxis]
    y_test = y_test[..., np.newaxis]
    y_validation = y_validation[..., np.newaxis]

    print("Dataset loaded!")
    
    
    return X_train, X_test, X_validation, y_train, y_test, y_validation

In [ ]:
def prepare_dataset(data_path):
    """Creates train, validation and test sets.
    """

    # load dataset
    X_train, X_test, X_validation, y_train, y_test, y_validation = load_data(data_path)
    
################# Scaleing the data ####################
    scaler = StandardScaler()
    num_instances, num_time_steps, num_features = X_train.shape
    X_train = X_train.reshape(-1, num_features)
    X_train = scaler.fit_transform(X_train)
    
    #reshapeing
    X_train = X_train.reshape(num_instances, num_time_steps, num_features) 
    num_instances, num_time_steps, num_features = X_test.shape
    X_test = X_test.reshape(-1, num_features)
    X_test = scaler.fit_transform(X_test)
    
    #reshapeing
    X_test = X_test.reshape(num_instances, num_time_steps, num_features) 

    num_instances, num_time_steps, num_features = X_validation.shape

    X_validation = X_validation.reshape(-1, num_features)
    X_validation = scaler.fit_transform(X_validation)
    
     #reshapeing
    X_validation = X_validation.reshape(num_instances, num_time_steps, num_features)  
    
    # Save the scaler to a file
    joblib.dump(scaler, './scaler/scaler.pkl')

    # add an axis to nd array
    X_train = X_train[..., np.newaxis]
    X_test = X_test[..., np.newaxis]
    X_validation = X_validation[..., np.newaxis]

    return X_train, y_train, X_validation, y_validation, X_test, y_test

In [ ]:
DATA_PATH = "/home/ec.gpu/Desktop/Soumen/kws/data_npy"
class_names = ['off', 'left', 'down', 'up', 'go', 'on', 'stop', 'unknown', 'right', 'yes']  #, 'silence' , 'no'
EPOCHS = 500
BATCH_SIZE = 64
PATIENCE = 5
LEARNING_RATE = 0.0001
SKIP = 1
CLASS = 10

In [ ]:
# generate train, validation and test sets
X_train, y_train, X_validation, y_validation, X_test, y_test = prepare_dataset(DATA_PATH)
print(X_train.shape)
print(y_train.shape)
print(X_validation.shape)
print(y_validation.shape)
print(X_test.shape)
print(y_test.shape)


# CNN

In [ ]:
from deap import base, creator, tools, algorithms
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
import random

results_1 = []

# Results to store specifications of models and their fitness values per generation
generation_pareto_fronts = []

In [ ]:
# Define custom crossover
def cxUniform(ind1, ind2, indpb):
    for i in range(len(ind1)):
        if random.random() < indpb:
            ind1[i], ind2[i] = ind2[i], ind1[i]
    return ind1, ind2

# Define custom mutation
def mutFlipBit(individual, indpb):
    for i in range(len(individual)):
        if random.random() < indpb:
            if isinstance(individual[i], bool):
                individual[i] = not individual[i]
            elif isinstance(individual[i], list):
                individual[i] = [random.choice([128, 256]) if isinstance(x, int) else x for x in individual[i]]
            elif isinstance(individual[i], tuple):
                individual[i] = (random.choice([3, 5]), random.choice([3, 5]))
            else:
                individual[i] = random.choice([16, 32, 64])
    return individual,


In [ ]:
# Create a CNN model 
def create_2d_cnn_model(conv_layers, filters, input_shape, kernel_size, fc_layers, use_bn, use_dropout):
    model = Sequential()
    model.add(Conv2D(filters, kernel_size=kernel_size, activation='relu', input_shape=input_shape, padding='same'))
    if use_bn:
        model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

    for _ in range(conv_layers - 1):
        model.add(Conv2D(filters, kernel_size=kernel_size, activation='relu', padding='same'))
        if use_bn:
            model.add(BatchNormalization())
        current_shape = model.output_shape
        if current_shape[1] > 2 and current_shape[2] > 2:
            model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    
    model.add(Flatten())

    for neurons in fc_layers:
        model.add(Dense(neurons, activation='relu'))
        if use_dropout:
            model.add(Dropout(0.5))

    model.add(Dense(CLASS, activation='softmax'))
    model.compile(optimizer='adam', loss='c', metrics=['accuracy'])
    return model

In [ ]:
# Train the model
def train(model, epochs, batch_size, patience, X_train, y_train, X_validation, y_validation):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)
    history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation),
                        epochs=epochs, batch_size=batch_size, callbacks=[early_stopping], verbose=0)
    return history


In [ ]:
# Evaluate the model and return accuracy and size
def evaluate_model(conv_layers, filters, kernel_size, fc_layers, use_bn, use_dropout):
   
    input_shape = (X_train.shape[1], X_train.shape[2], 1) 
    model = create_2d_cnn_model(conv_layers, filters, input_shape, kernel_size, fc_layers, use_bn, use_dropout)
    history = train(model, EPOCHS, BATCH_SIZE, PATIENCE, X_train, y_train, X_validation, y_validation)

    val_accuracy = model.evaluate(X_validation, y_validation, verbose=0)[1]
    model_size = model.count_params()
    results_1.append((conv_layers, filters, kernel_size, fc_layers, use_bn, use_dropout, val_accuracy, model_size))
    # Print the model summary
    # print(f"Layers: {conv_layers}, Filters: {filters}, Kernel: {kernel_size}, FC: {fc_layers} , BN: {use_bn}, Dropout: {use_dropout} - Val_Acc: {val_accuracy:.4f}, Params: {model_size}")
    #plot_history(history)
    
    return 1 - val_accuracy, model_size


In [ ]:
# Evaluation function
def evaluate(individual):
    conv_layers, filters, kernel_size, fc_layers, use_bn, use_dropout = individual
    return evaluate_model(conv_layers, filters, kernel_size, fc_layers, use_bn, use_dropout)


In [ ]:
# NSGA-II setup
creator.create("FitnessMulti", base.Fitness, weights=(-1.0, 1.0))
creator.create("Individual", list, fitness=creator.FitnessMulti)

toolbox = base.Toolbox()
toolbox.register("attr_conv_layers", np.random.randint, 1, 6)
toolbox.register("attr_filters", np.random.choice, [16, 32, 64])
toolbox.register("attr_kernel_size", lambda: (np.random.choice([3, 5]), np.random.choice([3, 5])))
toolbox.register("attr_fc_layers", lambda: [np.random.choice([128, 256, 512])])
toolbox.register("attr_use_bn", np.random.choice, [True, False])
toolbox.register("attr_use_dropout", np.random.choice, [True, False])

toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_conv_layers, toolbox.attr_filters, toolbox.attr_kernel_size,
                  toolbox.attr_fc_layers, toolbox.attr_use_bn, toolbox.attr_use_dropout),
                 n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", cxUniform, indpb=0.5)
toolbox.register("mutate", mutFlipBit, indpb=0.2)
toolbox.register("select", tools.selNSGA2)
toolbox.register("map", map)

# GA parameters
population_size = 30
num_generations =10
crossover_prob = 0.5
mutation_prob = 0.2

# Generate the population and run the algorithm
population = toolbox.population(n=population_size)

# Capture Pareto fronts for each generation
generation_pareto_fronts = []

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Function to capture Pareto front and model specifications at each generation
def capture_generation_pareto(population, gen_number, results_1):
    
    # Initialize costs as a list
    costs = []
    for result in results_1:
        conv_layers, filters, kernel_size, fc_layers, use_bn, use_dropout, val_accuracy, model_size = result
        costs.append((1 - val_accuracy, model_size))
    
    costs = np.array(costs)

    # Find Pareto-optimal models
    pareto_indices = is_pareto_efficient(costs)

    # Print Pareto-optimal results
    print("Pareto-optimal models:")
    for idx in pareto_indices:
        print(f"Configuration: {results_1[idx]}")
        
    # Convert results to DataFrame
    columns = ['Generation', 'Conv_Layers', 'Filters', 'Kernel_Size', 'FC_Layers', 'Use_BN', 'Use_Dropout', 'Validation_Accuracy', 'Model_Size']

    # DataFrame for all models
    df_all_models = pd.DataFrame(results_1, columns=columns[1:])
    df_all_models.insert(0, 'Generation', gen_number)

    # DataFrame for Pareto models
    pareto_models = [results_1[idx] for idx in pareto_indices]
    df_pareto = pd.DataFrame(pareto_models, columns=columns[1:])
    df_pareto.insert(0, 'Generation', gen_number)

    # Append to CSV files (create if they don't exist)
    df_all_models.to_csv('CNN/all_models.csv', mode='a', header=not pd.io.common.file_exists('CNN/all_models.csv'), index=False)
    df_pareto.to_csv('CNN/pareto_front.csv', mode='a', header=not pd.io.common.file_exists('CNN/pareto_front.csv'), index=False)
    
				
				# Plot Pareto frontier
    plt.figure(figsize=(12, 8))

    # Scatter plot for all models
    for i, result in enumerate(results_1):
        conv_layers, filters, kernel_size, fc_layers, use_bn, use_dropout, val_accuracy, model_size = result
        label = (f'Model {i+1}: {conv_layers} layers, {filters} filters, kernel {kernel_size}, '
                 f'fc {fc_layers}, BN: {use_bn}, Dropout: {use_dropout}')
        plt.scatter(model_size, val_accuracy, label=label, s=100)
        if i in pareto_indices:
            plt.annotate(f'{i+1}', (model_size, val_accuracy), textcoords="offset points", xytext=(0, 10), ha='center', color='red')

    # Extract Pareto frontier points
    pareto_points = costs[pareto_indices]

    # Plot Pareto frontier line
    if len(pareto_points) > 0:
        plt.plot(pareto_points[:, 1], 1 - pareto_points[:, 0], color='red', linestyle='--', linewidth=2, label='Pareto Frontier')

    plt.xlabel('Model Size (Number of Parameters)')
    plt.ylabel('Validation Accuracy')
    plt.title('Pareto Frontier: Model Size vs. Validation Accuracy')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.show()
    

In [ ]:
# Function to check Pareto efficiency
def is_pareto_efficient(costs):
    is_efficient = np.ones(costs.shape[0], dtype=bool)
    for i, c in enumerate(costs):
        if is_efficient[i]:
            is_efficient[is_efficient] = np.any(costs[is_efficient] < c, axis=1)
            is_efficient[i] = True
    return np.where(is_efficient)[0]

In [ ]:
# Main loop for running generations
for gen in range(num_generations):
    print(f"######## Generation: { gen } ###########")
    # Apply selection, crossover, and mutation
    offspring = toolbox.select(population, len(population))
    offspring = algorithms.varAnd(offspring, toolbox, cxpb=crossover_prob, mutpb=mutation_prob)
    
    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
    
    # Select the next generation population
    population[:] = toolbox.select(population + offspring, population_size)
    
    
    # Capture Pareto front for the current generation
    capture_generation_pareto(population, gen, results_1)
